In [9]:
import simpy
import numpy as np
import pandas as pd

# Defined Parameters
edge_datacenters = [
    {
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
] * 2

cloud_datacenters = [
    {
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
] * 2

class Server:
    def __init__(self, env, host_ram, num_hosts, storage, bandwidth, processing_power, vm_number, accumulated_ram, accumulated_storage, cost):
        self.env = env
        self.machine = simpy.Resource(env, capacity=num_hosts)
        self.latest_task_end_time = 0
        self.host_ram = host_ram
        self.storage = storage
        self.bandwidth = bandwidth
        self.processing_power = processing_power
        self.vm_number = vm_number
        self.accumulated_ram = accumulated_ram
        self.accumulated_storage = accumulated_storage
        self.cost = cost
        self.expected_end_time = 0
        self.total_processing_time = 0
        self.total_cost = 0

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout(task.duration)
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + task.duration
            self.total_processing_time += task.duration
            self.total_cost += self.cost * task.duration  # Calculate total cost

class Task:
    def __init__(self, duration):
        self.duration = duration

env = simpy.Environment()
MAX_COST = 350

class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {task_size: [] for task_size in tasks}
        self.average_cost = 0
        self.average_utilization = 0

    def __call__(self, solution):
        env = simpy.Environment()
        servers = self.servers  # Use the provided server objects
        task_objects = []

        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            task_objects.append(task)
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        # Calculate makespan as the maximum end time of all servers
        makespan = max(server.latest_task_end_time for server in servers)

        if makespan > 0:
            total_cost = sum(server.total_cost for server in servers)
            self.average_cost = total_cost / len(self.tasks)
            total_utilization = sum(server.total_processing_time / makespan for server in servers)
            self.average_utilization = total_utilization / len(self.servers)

            lambda_factor = 0.01  # hyperparameter that you can tune
            return makespan + lambda_factor * total_cost
        else:
            # Handle the case where makespan is zero (division by zero)
            return float('inf')

    def get_average_cost(self):
        return self.average_cost

    def get_average_utilization(self):
        return self.average_utilization

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.randint(
            low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf

class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, final_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.final_temperature = final_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.cooling_rate

class FlowerPollination:
    def __init__(self, fitness_function, population_size=100, decision_variables=10, lower_bound=0, upper_bound=1, generations=1000, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(
            decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * \
            (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(
            low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * \
            (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=100, final_temperature=0.001, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            #print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best

# Initialize edge servers and cloud servers with explicit parameters
edge_servers = [
    Server(env,
           host_ram=datacenter['host_ram'],
           num_hosts=datacenter['num_hosts'],
           storage=datacenter['storage'],
           bandwidth=datacenter['bandwidth'],
           processing_power=datacenter['processing_power'],
           vm_number=datacenter['vm_number'],
           accumulated_ram=datacenter['accumulated_ram'],
           accumulated_storage=datacenter['accumulated_storage'],
           cost=datacenter['cost']
           )
    for datacenter in edge_datacenters
] * 2

cloud_servers = [
    Server(env,
           host_ram=datacenter['host_ram'],
           num_hosts=datacenter['num_hosts'],
           storage=datacenter['storage'],
           bandwidth=datacenter['bandwidth'],
           processing_power=datacenter['processing_power'],
           vm_number=datacenter['vm_number'],
           accumulated_ram=datacenter['accumulated_ram'],
           accumulated_storage=datacenter['accumulated_storage'],
           cost=datacenter['cost']
           )
    for datacenter in cloud_datacenters
] * 2

task_sizes = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# Run your optimization for both cloud and edge servers.

# Initialize DataFrame
columns = ['Task Size', 'Server Type', 'Average Makespan', 'Average Cost', 'Average Utilization']
df = pd.DataFrame(columns=columns)

# Optimization and DataFrame appending loop
for task_size in task_sizes:
    # Optimization for cloud servers
    print(f"Optimizing for cloud servers with task size: {task_size}")
    fitness_function = FitnessFunction([task_size], cloud_servers)
    fp = FlowerPollination(fitness_function, population_size=20, decision_variables=1, lower_bound=0, upper_bound=len(cloud_servers) - 1, generations=100)
    best_flower = fp.optimize()
    average_cost = fitness_function.get_average_cost()
    average_utilization = fitness_function.get_average_utilization()
    df.loc[len(df)] = [task_size, 'Cloud', np.mean(fitness_function.makespans[task_size]), average_cost, average_utilization]

    # Optimization for edge servers
    print(f"Optimizing for edge servers with task size: {task_size}")
    fitness_function = FitnessFunction([task_size], edge_servers)
    fp = FlowerPollination(fitness_function, population_size=20, decision_variables=1, lower_bound=0, upper_bound=len(edge_servers) - 1, generations=100)
    best_flower = fp.optimize()
    average_cost = fitness_function.get_average_cost()
    average_utilization = fitness_function.get_average_utilization()
    df.loc[len(df)] = [task_size, 'Edge', np.mean(fitness_function.makespans[task_size]), average_cost, average_utilization]

    # Print the DataFrame after each optimization cycle
    print("Current DataFrame:")
    print(df)

# Save the DataFrame as a CSV file
df.to_csv('simulation_results.csv', index=False)


Optimizing for cloud servers with task size: 100


c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Optimizing for edge servers with task size: 100


c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Current DataFrame:
   Task Size Server Type  Average Makespan  Average Cost  Average Utilization
0        100       Cloud               NaN             0                    0
1        100        Edge               NaN             0                    0
Optimizing for cloud servers with task size: 200


...

c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Current DataFrame:
    Task Size Server Type  Average Makespan  Average Cost  Average Utilization
0         100       Cloud               NaN             0                    0
1         100        Edge               NaN             0                    0
2         200       Cloud               NaN             0                    0
3         200        Edge               NaN             0                    0
4         300       Cloud               NaN             0                    0
5         300        Edge               NaN             0                    0
6         400       Cloud               NaN             0                    0
7         400        Edge               NaN             0                    0
8         500       Cloud               NaN             0                    0
9         500        Edge               NaN             0                    0
10        600       Cloud               NaN             0                    0
11        600        Edge        

c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Optimizing for edge servers with task size: 1000
Current DataFrame:
    Task Size Server Type  Average Makespan  Average Cost  Average Utilization
0         100       Cloud               NaN             0                    0
1         100        Edge               NaN             0                    0
2         200       Cloud               NaN             0                    0
3         200        Edge               NaN             0                    0
4         300       Cloud               NaN             0                    0
5         300        Edge               NaN             0                    0
6         400       Cloud               NaN             0                    0
7         400        Edge               NaN             0                    0
8         500       Cloud               NaN             0                    0
9         500        Edge               NaN             0                    0
10        600       Cloud               NaN             0      

c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\YANSILIYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [3]:
import random
import math

# Define the parameters for data centers and tasks
edge_datacenters = [
    {
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
] * 2

cloud_datacenters = [
    {
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
] * 2

num_tasks_range = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
random.seed(42)

# Simulated Annealing parameters
initial_temperature = 1000
cooling_rate = 0.95
iterations_per_temperature = 100

def initial_solution(num_tasks, num_datacenters):
    # Generate an initial random task allocation
    return [random.randint(0, num_datacenters - 1) for _ in range(num_tasks)]

def evaluate_solution(task_allocation, datacenters):
    # Calculate the resource utilization for the given task allocation
    total_utilization = 0
    for task, datacenter_idx in enumerate(task_allocation):
        datacenter = datacenters[datacenter_idx]
        total_utilization += (datacenter['accumulated_ram'] + datacenter['accumulated_storage']) / (datacenter['host_ram'] + datacenter['storage'])
    return total_utilization

def neighbor_solution(current_solution, num_datacenters):
    # Generate a neighboring solution by moving a random task to a different data center
    neighbor = current_solution.copy()
    task_to_move = random.randint(0, len(neighbor) - 1)
    new_datacenter = random.randint(0, num_datacenters - 1)
    neighbor[task_to_move] = new_datacenter
    return neighbor

def simulated_annealing(num_tasks, datacenters):
    current_solution = initial_solution(num_tasks, len(datacenters))
    current_utilization = evaluate_solution(current_solution, datacenters)
    best_solution = current_solution
    best_utilization = current_utilization
    temperature = initial_temperature

    while temperature > 1:
        for _ in range(iterations_per_temperature):
            neighbor = neighbor_solution(current_solution, len(datacenters))
            neighbor_utilization = evaluate_solution(neighbor, datacenters)
            delta_utilization = neighbor_utilization - current_utilization

            if delta_utilization > 0 or random.random() < math.exp(delta_utilization / temperature):
                current_solution = neighbor
                current_utilization = neighbor_utilization

                if current_utilization > best_utilization:
                    best_solution = current_solution
                    best_utilization = current_utilization

        temperature *= cooling_rate

    return best_solution, best_utilization

# Run the simulated annealing for different numbers of tasks
for num_tasks in num_tasks_range:
    best_allocation, best_utilization = simulated_annealing(num_tasks, edge_datacenters)
    print(f"Number of tasks: {num_tasks}, Best Utilization: {best_utilization}")



Number of tasks: 100, Best Utilization: 9.523809523809508
Number of tasks: 200, Best Utilization: 19.047619047618998
Number of tasks: 300, Best Utilization: 28.571428571428488
Number of tasks: 400, Best Utilization: 38.09523809523798
Number of tasks: 500, Best Utilization: 47.61904761904747
Number of tasks: 600, Best Utilization: 57.14285714285696
Number of tasks: 700, Best Utilization: 66.66666666666664
Number of tasks: 800, Best Utilization: 76.19047619047684
Number of tasks: 900, Best Utilization: 85.71428571428704
Number of tasks: 1000, Best Utilization: 95.23809523809724


In [5]:
import random
import math
import time

# Define the parameters for data centers and tasks
edge_datacenters = [
    {
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
] * 2

cloud_datacenters = [
    {
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
] * 2

num_tasks_range = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
random.seed(42)

# Simulated Annealing parameters
initial_temperature = 1000
cooling_rate = 0.95
iterations_per_temperature = 100

def initial_solution(num_tasks, num_datacenters):
    # Generate an initial random task allocation
    return [random.randint(0, num_datacenters - 1) for _ in range(num_tasks)]

def evaluate_solution(task_allocation, datacenters):
    # Calculate the resource utilization for the given task allocation
    total_utilization = 0
    total_cost = 0
    for task, datacenter_idx in enumerate(task_allocation):
        datacenter = datacenters[datacenter_idx]
        total_utilization += (datacenter['accumulated_ram'] + datacenter['accumulated_storage']) / (datacenter['host_ram'] + datacenter['storage'])
        total_cost += datacenter['cost']
    return total_utilization, total_cost

def neighbor_solution(current_solution, num_datacenters):
    # Generate a neighboring solution by moving a random task to a different data center
    neighbor = current_solution.copy()
    task_to_move = random.randint(0, len(neighbor) - 1)
    new_datacenter = random.randint(0, num_datacenters - 1)
    neighbor[task_to_move] = new_datacenter
    return neighbor

def simulated_annealing(num_tasks, datacenters):
    current_solution = initial_solution(num_tasks, len(datacenters))
    current_utilization, current_cost = evaluate_solution(current_solution, datacenters)
    best_solution = current_solution
    best_utilization, best_cost = current_utilization, current_cost
    temperature = initial_temperature

    start_time = time.time()  # Record the start time

    while temperature > 1:
        for _ in range(iterations_per_temperature):
            neighbor = neighbor_solution(current_solution, len(datacenters))
            neighbor_utilization, neighbor_cost = evaluate_solution(neighbor, datacenters)
            delta_utilization = neighbor_utilization - current_utilization

            if delta_utilization > 0 or random.random() < math.exp(delta_utilization / temperature):
                current_solution = neighbor
                current_utilization = neighbor_utilization
                current_cost = neighbor_cost

                if current_utilization > best_utilization:
                    best_solution = current_solution
                    best_utilization = current_utilization
                    best_cost = current_cost

        temperature *= cooling_rate

    end_time = time.time()  # Record the end time

    makespan = end_time - start_time  # Calculate the makespan

    return best_solution, best_utilization, best_cost, makespan

# Run the simulated annealing for different numbers of tasks
for num_tasks in num_tasks_range:
    total_makespan = 0
    total_cost = 0
    total_utilization = 0

    for _ in range(10):  # Repeat the simulation 10 times for each number of tasks and calculate the average
        best_allocation, best_utilization, best_cost, makespan = simulated_annealing(num_tasks, edge_datacenters)
        total_utilization += best_utilization
        total_cost += best_cost
        total_makespan += makespan

    average_utilization = total_utilization / 10
    average_cost = total_cost / 10
    average_makespan = total_makespan / 10

    print(f"Number of tasks: {num_tasks}")
    print(f"Average Utilization: {average_utilization}")
    print(f"Average Cost: {average_cost}")
    print(f"Average Makespan: {average_makespan}\n")

Number of tasks: 100
Average Utilization: 9.523809523809506
Average Cost: 2.1999999999999993
Average Makespan: 0.32879080772399905

Number of tasks: 200
Average Utilization: 19.047619047619
Average Cost: 4.399999999999986
Average Makespan: 0.6157603740692139

Number of tasks: 300
Average Utilization: 28.57142857142849
Average Cost: 6.60000000000001
Average Makespan: 0.9538337230682373

Number of tasks: 400
Average Utilization: 38.095238095237974
Average Cost: 8.800000000000036
Average Makespan: 1.3255715370178223

Number of tasks: 500
Average Utilization: 47.61904761904747
Average Cost: 11.000000000000059
Average Makespan: 1.792346143722534

Number of tasks: 600
Average Utilization: 57.14285714285696
Average Cost: 13.200000000000085
Average Makespan: 2.2128928184509276

Number of tasks: 700
Average Utilization: 66.66666666666664
Average Cost: 15.400000000000109
Average Makespan: 2.534932589530945

Number of tasks: 800
Average Utilization: 76.19047619047684
Average Cost: 17.599999999999

In [9]:
import random
import math
import time

# Define the parameters for data centers and tasks
edge_datacenters = [
    {
        'host_ram': 10,
        'num_hosts': 1,
        'storage': 0.5,
        'bandwidth': 5,
        'processing_power': 250000,
        'vm_number': 30,
        'accumulated_ram': 0.5,
        'accumulated_storage': 0.5,
        'cost': 0.022,
    }
] * 2

cloud_datacenters = [
    {
        'host_ram': 20,
        'num_hosts': 1,
        'storage': 1,
        'bandwidth': 10,
        'processing_power': 1000000,
        'vm_number': 30,
        'accumulated_ram': 2,
        'accumulated_storage': 10,
        'cost': 0.051,
    }
] * 2

num_tasks_range = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
random.seed(42)

# Simulated Annealing parameters
initial_temperature = 1000
cooling_rate = 0.95
iterations_per_temperature = 100

def initial_solution(num_tasks, num_datacenters):
    # Generate an initial random task allocation
    return [random.randint(0, num_datacenters - 1) for _ in range(num_tasks)]

def evaluate_solution(task_allocation, datacenters):
    # Calculate the resource utilization for the given task allocation
    total_utilization = 0
    total_cost = 0
    for task, datacenter_idx in enumerate(task_allocation):
        datacenter = datacenters[datacenter_idx]
        total_utilization += (datacenter['accumulated_ram'] + datacenter['accumulated_storage']) / (datacenter['host_ram'] + datacenter['storage'])
        total_cost += datacenter['cost']
    return total_utilization, total_cost

def neighbor_solution(current_solution, num_datacenters):
    # Generate a neighboring solution by moving a random task to a different data center
    neighbor = current_solution.copy()
    task_to_move = random.randint(0, len(neighbor) - 1)
    new_datacenter = random.randint(0, num_datacenters - 1)
    neighbor[task_to_move] = new_datacenter
    return neighbor

# Flower Pollination parameters
pollination_rate = 0.2

def flower_pollination(current_solution, num_datacenters):
    new_solution = current_solution.copy()

    num_perturb_tasks = int(pollination_rate * len(new_solution))

    for _ in range(num_perturb_tasks):
        task_to_perturb = random.randint(0, len(new_solution) - 1)
        new_datacenter = random.randint(0, num_datacenters - 1)
        new_solution[task_to_perturb] = new_datacenter

    return new_solution

# Simulated Annealing with Flower Pollination
def simulated_annealing_with_fp(num_tasks, datacenters):
    current_solution = initial_solution(num_tasks, len(datacenters))
    current_utilization, current_cost = evaluate_solution(current_solution, datacenters)
    best_solution = current_solution
    best_utilization, best_cost = current_utilization, current_cost
    temperature = initial_temperature

    start_time = time.time()

    while temperature > 1:
        for _ in range(iterations_per_temperature):
            perturbed_solution = flower_pollination(current_solution, len(datacenters))
            perturbed_utilization, perturbed_cost = evaluate_solution(perturbed_solution, datacenters)
            delta_utilization = perturbed_utilization - current_utilization

            if delta_utilization > 0 or random.random() < math.exp(delta_utilization / temperature):
                current_solution = perturbed_solution
                current_utilization = perturbed_utilization
                current_cost = perturbed_cost

                if current_utilization > best_utilization:
                    best_solution = current_solution
                    best_utilization = current_utilization
                    best_cost = current_cost

        temperature *= cooling_rate

    end_time = time.time()
    makespan = end_time - start_time

    return best_solution, best_utilization, best_cost, makespan

# Run the simulated annealing with flower pollination for edge and cloud data centers separately
def run_simulation(datacenters, datacenter_name):
    for num_tasks in num_tasks_range:
        total_makespan = 0
        total_cost = 0
        total_utilization = 0

        for _ in range(10):
            best_allocation, best_utilization, best_cost, makespan = simulated_annealing_with_fp(num_tasks, datacenters)
            total_utilization += best_utilization
            total_cost += best_cost
            total_makespan += makespan

        average_utilization = total_utilization / 10
        average_cost = total_cost / 10
        average_makespan = total_makespan / 10

        print(f"{datacenter_name} Data Center:")
        print(f"Number of tasks: {num_tasks}")
        print(f"Average Utilization: {average_utilization}")
        print(f"Average Cost: {average_cost}")
        print(f"Average Makespan: {average_makespan}\n")

# Run the simulation for both Edge and Cloud data centers
run_simulation(edge_datacenters, "Edge")
run_simulation(cloud_datacenters, "Cloud")

Edge Data Center:
Number of tasks: 100
Average Utilization: 9.523809523809506
Average Cost: 2.1999999999999993
Average Makespan: 0.7115299224853515

Edge Data Center:
Number of tasks: 200
Average Utilization: 19.047619047619
Average Cost: 4.399999999999986
Average Makespan: 1.344028902053833

Edge Data Center:
Number of tasks: 300
Average Utilization: 28.57142857142849
Average Cost: 6.60000000000001
Average Makespan: 2.221557927131653

Edge Data Center:
Number of tasks: 400
Average Utilization: 38.095238095237974
Average Cost: 8.800000000000036
Average Makespan: 3.090134310722351

Edge Data Center:
Number of tasks: 500
Average Utilization: 47.61904761904747
Average Cost: 11.000000000000059
Average Makespan: 3.8287923097610475

Edge Data Center:
Number of tasks: 600
Average Utilization: 57.14285714285696
Average Cost: 13.200000000000085
Average Makespan: 4.7643776655197145

Edge Data Center:
Number of tasks: 700
Average Utilization: 66.66666666666664
Average Cost: 15.400000000000109
Ave